In [12]:
import numpy as np
import pandas as pd
import random
import os

In [13]:
df = pd.read_csv('train_dataset.csv')
print('共{}个用户，{}本图书，{}条记录'.format(max(df['user_id'])+1, max(df['item_id'])+1, len(df)))

共53424个用户，10000本图书，5869631条记录


In [14]:
df.head()

,user_id,item_id
0,0,257
1,0,267
2,0,5555
3,0,3637
4,0,1795


## 数据预处理¶
构建Dataset类
构建负样本
划分测试集与验证集
构建对应的Dataloader

In [15]:
rows_count_per_id = df.groupby('user_id').size().reset_index(name='count')
rows_count_per_id['count'].min()

17

In [16]:
def get_sequence_and_next_value(data):
    window_size=10
    sequences = []
    next_values = []
    data=data.item_id.values
    
    for i in range(len(data) - window_size):
        sequence = data[i : i + window_size]
        next_value = data[i + window_size]
        sequences.append(sequence)
        next_values.append(next_value)
  
    # 生成的序列和对应值
    data = {
        'Sequence': sequences,
        'Next_Value': next_values
    }
    
    # 将数据转换为 DataFrame
    df = pd.DataFrame(data)
    
    return df
    

# 将数据转换为列表


In [17]:
x=df.groupby('user_id').apply(get_sequence_and_next_value)

In [18]:
x=x.reset_index(drop=True)

x=x.sample(n=100000)

In [19]:
X=x.Sequence.values
df=np.vstack(X)
df=pd.DataFrame(df)
Y=x.Next_Value.values

In [20]:
Y

array([2961,  459,  324, ..., 5980,   26, 4463], dtype=int64)

In [21]:
from pycaret.regression import *
s = setup( df, target=Y, session_id = 123)


,Description,Value
0,Session id,123
1,Target,target
2,Target type,Regression
3,Original data shape,"(100000, 11)"
4,Transformed data shape,"(100000, 11)"
5,Transformed train set shape,"(70000, 11)"
6,Transformed test set shape,"(30000, 11)"
7,Numeric features,10
8,Preprocess,True
9,Imputation type,simple


In [ ]:
rf=s.create_model('br')

In [49]:
df = pd.read_csv('train_dataset.csv')
rows_count_per_id = df.groupby('user_id').tail(10)

In [38]:
def pre(rows_count_per_id):
    
    xx=rf.predict(rows_count_per_id.item_id.values.reshape(1, -1))
    return int(xx)
    return  xx

In [50]:
rows_count_per_id = rows_count_per_id.groupby('user_id').apply(pre)

In [53]:
rows_count_per_id.values

array([1309,  971, 1358, ...,  993, 1867, 1234], dtype=int64)

In [62]:
df=pd.DataFrame()
df['user_id']=rows_count_per_id.index
df['item_id']=rows_count_per_id.values
df.to_csv('submission1.csv', index=False) 

In [58]:
df

,user_id,item_id
0,0,1309
1,1,971
2,2,1358
3,3,1329
4,4,3189
...,...,...
53419,53419,3111
53420,53420,5292
53421,53421,993
53422,53422,1867


In [64]:
rf=s.create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1620.8069,4857456.7985,2203.9639,0.2013,1.8780,17.5517
1,1590.1587,4717379.8948,2171.9530,0.2221,1.8607,18.1096
2,1615.8981,4849096.1118,2202.0663,0.2056,1.8440,16.2047
3,1634.7051,4945877.1845,2223.9328,0.1810,1.8771,17.5570
4,1611.9254,4895347.0434,2212.5431,0.1989,1.8679,17.6085
5,1623.8324,4997279.1891,2235.4595,0.1844,1.8613,21.1533
6,1639.5268,5020074.4642,2240.5523,0.2045,1.8577,17.4994
7,1617.9852,4836597.7674,2199.2266,0.1837,1.9005,18.9007
8,1621.5093,4907618.1691,2215.3145,0.2060,1.8899,18.4654


In [65]:
df = pd.read_csv('train_dataset.csv')
rows_count_per_id = df.groupby('user_id').tail(10)
rows_count_per_id = rows_count_per_id.groupby('user_id').apply(pre)
df=pd.DataFrame()
df['user_id']=rows_count_per_id.index
df['item_id']=rows_count_per_id.values
df.to_csv('submission1.csv', index=False) 